In [5]:
import numpy as np
import cv2
from pupil_apriltags import Detector

# Load the calibration results from the .npz file
calibration_data = np.load('C:/Users/ifeda/OneDrive/Pictures/Documents/Fall 2023/Robot Perception/HW1/calibration_result.npz')

# Access the saved variables (K matrix, distortion coefficients, etc.) by their names
K_matrix = calibration_data['mtx']
distortion_coefficients = calibration_data['dist'].reshape((-1,1))
rotation_vectors = calibration_data['rvecs']
translation_vectors = calibration_data['tvecs']


# Open a connection to your webcam (usually 0 for the default webcam)
fx = K_matrix[0][0]
fy = K_matrix[1][1]
cx = K_matrix[0][2]
cy = K_matrix[1][2]


# Initialize the AprilTag detector
at_detector = Detector(
   families="tag36h11",
   nthreads=1,
   quad_decimate=1.0,
   quad_sigma=0.0,
   refine_edges=1,
   decode_sharpening=0.25,
   debug=0
)

cap = cv2.VideoCapture(0)
image_count = 0
while True:
    # Capture a frame from the webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect AprilTags in the grayscale frame
    tags = at_detector.detect(img = gray, estimate_tag_pose = True, camera_params = (fx, fy, cx, cy), tag_size =0.034)
    if len(tags) > 0:
        for tag in tags:
            # Get the tag's pose
            pose_trans = tag.pose_t.squeeze()
            pose_rotat = tag.pose_R

            # Define cube vertices in the tag's local coordinate system
            tag_size = 0.034
            half_size = tag_size / 2
            cube_vertices = np.array([
                [-half_size, -half_size, 0],
                [half_size, -half_size, 0],
                [half_size, half_size, 0],
                [-half_size, half_size, 0],
                [-half_size, -half_size, -tag_size],
                [half_size, -half_size, -tag_size],
                [half_size, half_size, -tag_size],
                [-half_size, half_size, -tag_size]
            ])



            # Project the 3D cube vertices to 2D image coordinates
            cube_2d, _ = cv2.projectPoints(cube_vertices, pose_rotat, pose_trans, K_matrix, distortion_coefficients)
            # Draw lines connecting the cube vertices on the image
            cube_2d = cube_2d.reshape(-1, 2)

            for i in range(4):
                cv2.line(frame, tuple(map(int, cube_2d[i])), tuple(map(int, cube_2d[(i + 1) % 4])), (0, 255, 0), 2)
                cv2.line(frame, tuple(map(int, cube_2d[i + 4])), tuple(map(int, cube_2d[(i + 1) % 4 + 4])), (0, 255, 0), 2)
                cv2.line(frame, tuple(map(int, cube_2d[i])), tuple(map(int, cube_2d[i + 4])), (0, 255, 0), 2)
                
            image_filename = f"C:/Users/ifeda/OneDrive/Pictures/Documents/Fall 2023/Robot Perception/HW1/Q4_Output/image_{image_count}.jpg"
            cv2.imshow('AR Cube on AprilTag', frame)
            cv2.imwrite(image_filename, frame)
            print(f"Saved image as {image_filename}")
            image_count += 1
    # Display the frame with detected tags and the rendered 3D cubes
    cv2.imshow('AR Cube on AprilTag', frame)

    # Exit the loop when the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the OpenCV window
cap.release()
cv2.destroyAllWindows()

Saved image as C:/Users/ifeda/OneDrive/Pictures/Documents/Fall 2023/Robot Perception/HW1/Q4_Output/image_0.jpg
Saved image as C:/Users/ifeda/OneDrive/Pictures/Documents/Fall 2023/Robot Perception/HW1/Q4_Output/image_1.jpg
Saved image as C:/Users/ifeda/OneDrive/Pictures/Documents/Fall 2023/Robot Perception/HW1/Q4_Output/image_2.jpg
Saved image as C:/Users/ifeda/OneDrive/Pictures/Documents/Fall 2023/Robot Perception/HW1/Q4_Output/image_3.jpg
Saved image as C:/Users/ifeda/OneDrive/Pictures/Documents/Fall 2023/Robot Perception/HW1/Q4_Output/image_4.jpg
Saved image as C:/Users/ifeda/OneDrive/Pictures/Documents/Fall 2023/Robot Perception/HW1/Q4_Output/image_5.jpg
Saved image as C:/Users/ifeda/OneDrive/Pictures/Documents/Fall 2023/Robot Perception/HW1/Q4_Output/image_6.jpg
Saved image as C:/Users/ifeda/OneDrive/Pictures/Documents/Fall 2023/Robot Perception/HW1/Q4_Output/image_7.jpg
Saved image as C:/Users/ifeda/OneDrive/Pictures/Documents/Fall 2023/Robot Perception/HW1/Q4_Output/image_8.jpg
S

In [8]:
cube_2d

array([[502.29818619, 319.64509373],
       [429.18723933, 315.24304399],
       [435.64497787, 247.04386857],
       [506.47265406, 247.62491959],
       [487.01689898, 300.90939374],
       [406.19007777, 297.19824097],
       [414.12902455, 222.19522227],
       [492.12232556, 221.21732325]])